In [ ]:
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org pycryptodome

In [ ]:
#!pip install /home/naidd009/Downloads/botocore-1.21.58.tar.gz --trusted-host pypi.org --trusted-host files.pythonhosted.org

In [ ]:
#!pip install boto3 --trusted-host pypi.org --trusted-host files.pythonhosted.org

# INITIALISE

In [1]:

import os
from datetime import datetime
from Crypto.Cipher import AES
from Crypto.Util import Padding
import pandas as pd
# will be using CBC mode which requires an IV. A static IV will be created together with the secret key
class Encrypt(object):

    def __init__(self):
        self.MULTIPLE_OF_SIXTEEN = 16

    def generate_secret(self,path=''):
        """
        python 3 is nice with from binary .hex() and from hex to bytes .fromhex()
        """
        secret = os.urandom(self.MULTIPLE_OF_SIXTEEN).hex()
        output_file = path+"secret_hex_" + datetime.now().strftime('%Y-%m-%d-%H-%M-%S') + '.secret'
        with open(output_file, "w") as file:
            # We don't need write in binary mode anymore
            file.write(secret)
            
        IV = AES.new(secret.encode(),AES.MODE_CBC).iv
        
        iv_file_name_str = output_file.replace('.secret','.bin')
        
        f = open(iv_file_name_str, "wb")
        f.write(IV)
        f.close()

        return secret,IV,output_file

    @staticmethod
    def get_secret(secret_file_name_str):
        """
        We expect a hex encoded secret
        """
        with open(secret_file_name_str, "r") as file:
            # we don't need to read in binary mode here
            secret = file.read()
        iv_file_name_str = secret_file_name_str.replace('.secret','.bin')
        f = open(iv_file_name_str, "rb")
        IV = f.read()
        f.close()
        return secret,IV

    @staticmethod
    def get_closest_larger_multiple_of_sixteen(n):
        remainder = n % 16
        if remainder == 0:
            return n
        else:
            return n + (16 - remainder)

    def encrypt_df_columns(self, df: pd.DataFrame, list_of_columns:list, secret_file_name_str=None):
        df2 = df.copy()
        if secret_file_name_str == None:
            secret,IV,secret_file_name_str = self.generate_secret()
        else:
            secret,IV = self.get_secret(secret_file_name_str)
           
        for col in list_of_columns:
            try:
                # make the encryption object
                #obj = AES.new(secret_pad,AES.MODE_SIV)
                df2[col] = df2[col].apply(
                    lambda x : AES.new(secret.encode(),AES.MODE_CBC,iv=IV).encrypt( Padding.pad(str(x).encode(),16) )
                                            )
                # above tries to do "smart padding" by expanding to a multiple of 16 only if needed
            except:
                print(f"error: {col}")
                pass

        return df2

    def decrypt_df_columns(self,df:pd.DataFrame, list_of_columns:list, list_of_dtypes:list,secret_file_name_str:str):
        df2 = df.copy()
        if secret_file_name_str == None:
            raise ValueError('Aborting -- we need a secret file to continue decrypt')
        else:
            secret,IV = self.get_secret(secret_file_name_str)

        for col,typ in zip(list_of_columns,list_of_dtypes):
            try:
                df2[col] = df2[col].apply(lambda x: Padding.unpad(AES.new(secret.encode()
                                                                          ,AES.MODE_CBC,iv=IV).decrypt(x),16).decode())
                df2[col] = df2[col].astype(typ)
            except:
                print(f'error: {col}')
                pass
        return df2

    def hash_df_columns(self,df:pd.DataFrame, list_of_columns:list):
        for col in list_of_columns:
            df[col] = df[col].apply(lambda x: hash(str(x)))
        return df


In [3]:
import cx_Oracle
def Mpesa_Test(dsn,username,password,query):

    """ Connect to the database"""
    
    try:

        connection = cx_Oracle.connect(username, password, dsn)

        print (connection.version)

    except cx_Oracle.DatabaseError as errror_detail:

        print('Connection failed due to:',errror_detail)

        #raise


    cursor = connection.cursor()

    cursor.arraysize = 2500

    try:

        cursor.execute(query)

        allRows = []

        global names

        names = [ x[0] for x in cursor.description]

        for row in cursor.fetchall():               

            allRows.append(row)

        return pd.DataFrame( allRows, columns=names)

    except cx_Oracle.DatabaseError as err_detail:

        print('Query failed to run due to :',err_detail)

            #raise


# IMPORT DATA FROM SQL

In [13]:
#INITIALISE CONNECTION AND IMPORT DATA
username = 'NAIDD009'
#username = 'panda008'

#Import MIS Password from local file
mis_pass = open('mis_pass.txt', "r")
password = mis_pass.read()

dsn = cx_Oracle.makedsn('10.200.200.54',1521,'pmis01').replace('SID','SERVICE_NAME')

#GET SAMPLE OF CLIENT DATA
query =  f"""                     
       select * from ODS.FCT_MMAE_MM_ACCOUNT_ENTRY where rownum < 51
                            """

df = Mpesa_Test(dsn,username,password,query)

11.2.0.4.0


In [14]:
df.head()

F_FI_ID   MMAE_LOAD_DATA_TS  MMAE_ACCOUNT_ENTRY_INDEX MMAE_TRANS_INDEX  \
0  763758329 2020-03-16 11:09:53                3887468029       7CD0RQEYOA   
1  763758329 2020-03-16 11:09:53                3887468038       7CD4RQEYOE   
2  763758329 2020-03-16 11:09:53                3887463928       7CD0RQEZAI   
3  763758329 2020-03-16 11:09:53                3887466508       7CD9RQF00B   
4  763758329 2020-03-16 11:09:53                3887466509       7CD9RQF00B   

  MMAE_RECORDTIMESTAMP REF_FT_ID MMAE_BUSINESS_SCOPE  F_RT_SEQID  REF_RT_ID  \
0  2020-03-13 08:27:32         2                   2         733       1237   
1  2020-03-13 08:27:33         2                   2         743       1278   
2  2020-03-13 08:27:33         1                   2         743       1278   
3  2020-03-13 08:27:33         1                   2         743       1278   
4  2020-03-13 08:27:33         2                   2         743       1278   

   REF_CT_ID  ...              MMAE_TRANS_IDENTITY_MNEMONIC  \
0          2  ...           243818202547 - Francine Kapinga   
1          2  ...  243820915425 - ROSETTE BATAKAMBA NTAMBUE   
2         29  ...                            3218 - ETS MKA   
3         29  ...       5137203 - YEMBA ONGENGE RARE5137203   
4          2  ...       5137203 - YEMBA ONGENGE RARE5137203   

  MMAE_ACCOUNT_VERSION MMAE_TRANSACTION_END_TIME MMAE_SUBJECT_ID  \
0                 1582       2020-03-13 08:27:33               0   
1                  370       2020-03-13 08:27:33               0   
2              1473965       2020-03-13 08:27:33               0   
3                 5830       2020-03-13 08:27:33               0   
4                 5864       2020-03-13 08:27:33               0   

   MMAE_AE_CREATE_TIME  MMAE_AE_AUDIT_INDEX  MMAE_RESERVE_BALANCE  \
0  2020-03-13 08:27:33           2923895500                     0   
1  2020-03-13 08:27:33           2923895526                     0   
2  2020-03-13 08:27:33           2923895532                     0   
3  2020-03-13 08:27:33           2923895538                     0   
4  2020-03-13 08:27:33           2923895539                     0   

   MMAE_PRE_RESERVE_BALANCE MMAE_PRE_AVAILABLE_BALANCE  F_FI_ID_ORIG  
0                         0                     676890            -1  
1                         0                     999687            -1  
2                         0                    2419243            -1  
3                         0                      64586            -1  
4                         0                     258415            -1  

[5 rows x 40 columns]

In [17]:
df[['MMAE_IDENTITY_MNEMONIC','MMAE_TRANS_IDENTITY_MNEMONIC','MMAE_DETAILS']].head()

MMAE_IDENTITY_MNEMONIC  \
0           243818202547 - Francine Kapinga   
1  243820915425 - ROSETTE BATAKAMBA NTAMBUE   
2                            3218 - ETS MKA   
3       5137203 - YEMBA ONGENGE RARE5137203   
4       5137203 - YEMBA ONGENGE RARE5137203   

               MMAE_TRANS_IDENTITY_MNEMONIC  \
0           243818202547 - Francine Kapinga   
1  243820915425 - ROSETTE BATAKAMBA NTAMBUE   
2                            3218 - ETS MKA   
3       5137203 - YEMBA ONGENGE RARE5137203   
4       5137203 - YEMBA ONGENGE RARE5137203   

                                    MMAE_DETAILS  
0                243818202547 - Francine Kapinga  
1                         Deposit at Agent Store  
2  Deposit Commission received from Charge Payer  
3  Deposit Commission received from Charge Payer  
4                         Deposit at Agent Store

# ENCRYPT PII COLUMNS

In [12]:
# GENERATE SECRET KEY
# YOU CAN JUST RUN THIS ONCE AND STORE THE KEYS SOMEWHERE SAFE
encrypt = Encrypt()

encrypt.generate_secret('../main_keys/')

('6cda58ae2cceb76e75454987a3e31a91',
 b'{\xc0\x91\xd1\x9d\xf3%\x0b\xb2S>\xe7V\x9aV\xbb',
 '../main_keys/secret_hex_2021-10-28-11-18-48.secret')

In [18]:
# ENCRYPT DATA
import pyarrow
df_encrypt = encrypt.encrypt_df_columns(df,['MMAE_IDENTITY_MNEMONIC','MMAE_TRANS_IDENTITY_MNEMONIC','MMAE_DETAILS'],'../main_keys/secret_hex_2021-10-28-11-18-48.secret')
df_encrypt.to_parquet('drc_acct_entry_50.parquet',index=False)

In [19]:
df_encrypt.head()

F_FI_ID   MMAE_LOAD_DATA_TS  MMAE_ACCOUNT_ENTRY_INDEX MMAE_TRANS_INDEX  \
0  763758329 2020-03-16 11:09:53                3887468029       7CD0RQEYOA   
1  763758329 2020-03-16 11:09:53                3887468038       7CD4RQEYOE   
2  763758329 2020-03-16 11:09:53                3887463928       7CD0RQEZAI   
3  763758329 2020-03-16 11:09:53                3887466508       7CD9RQF00B   
4  763758329 2020-03-16 11:09:53                3887466509       7CD9RQF00B   

  MMAE_RECORDTIMESTAMP REF_FT_ID MMAE_BUSINESS_SCOPE  F_RT_SEQID  REF_RT_ID  \
0  2020-03-13 08:27:32         2                   2         733       1237   
1  2020-03-13 08:27:33         2                   2         743       1278   
2  2020-03-13 08:27:33         1                   2         743       1278   
3  2020-03-13 08:27:33         1                   2         743       1278   
4  2020-03-13 08:27:33         2                   2         743       1278   

   REF_CT_ID  ...                       MMAE_TRANS_IDENTITY_MNEMONIC  \
0          2  ...  b'\x0c\xfa#|\x9b\xc0;\x16\xaf\xa5\x8f.\x95\xad...   
1          2  ...  b'\xc8\xedY\x92\x1dmK\xc0y6}\xa8\xcb\xaacqe\xb...   
2         29  ...  b'*\x17\xa9g\xfdC\xca\x0e3\x115\xa0N\xf3\x85\x02'   
3         29  ...  b'\xa4\x88\x1a(\x99tU\xff\xf60#\xdd\xd5x\x1e\x...   
4          2  ...  b'\xa4\x88\x1a(\x99tU\xff\xf60#\xdd\xd5x\x1e\x...   

  MMAE_ACCOUNT_VERSION MMAE_TRANSACTION_END_TIME MMAE_SUBJECT_ID  \
0                 1582       2020-03-13 08:27:33               0   
1                  370       2020-03-13 08:27:33               0   
2              1473965       2020-03-13 08:27:33               0   
3                 5830       2020-03-13 08:27:33               0   
4                 5864       2020-03-13 08:27:33               0   

   MMAE_AE_CREATE_TIME  MMAE_AE_AUDIT_INDEX  MMAE_RESERVE_BALANCE  \
0  2020-03-13 08:27:33           2923895500                     0   
1  2020-03-13 08:27:33           2923895526                     0   
2  2020-03-13 08:27:33           2923895532                     0   
3  2020-03-13 08:27:33           2923895538                     0   
4  2020-03-13 08:27:33           2923895539                     0   

   MMAE_PRE_RESERVE_BALANCE MMAE_PRE_AVAILABLE_BALANCE  F_FI_ID_ORIG  
0                         0                     676890            -1  
1                         0                     999687            -1  
2                         0                    2419243            -1  
3                         0                      64586            -1  
4                         0                     258415            -1  

[5 rows x 40 columns]

In [20]:
df_encrypt[['MMAE_IDENTITY_MNEMONIC','MMAE_TRANS_IDENTITY_MNEMONIC','MMAE_DETAILS']].head()

MMAE_IDENTITY_MNEMONIC  \
0  b'\x0c\xfa#|\x9b\xc0;\x16\xaf\xa5\x8f.\x95\xad...   
1  b'\xc8\xedY\x92\x1dmK\xc0y6}\xa8\xcb\xaacqe\xb...   
2  b'*\x17\xa9g\xfdC\xca\x0e3\x115\xa0N\xf3\x85\x02'   
3  b'\xa4\x88\x1a(\x99tU\xff\xf60#\xdd\xd5x\x1e\x...   
4  b'\xa4\x88\x1a(\x99tU\xff\xf60#\xdd\xd5x\x1e\x...   

                        MMAE_TRANS_IDENTITY_MNEMONIC  \
0  b'\x0c\xfa#|\x9b\xc0;\x16\xaf\xa5\x8f.\x95\xad...   
1  b'\xc8\xedY\x92\x1dmK\xc0y6}\xa8\xcb\xaacqe\xb...   
2  b'*\x17\xa9g\xfdC\xca\x0e3\x115\xa0N\xf3\x85\x02'   
3  b'\xa4\x88\x1a(\x99tU\xff\xf60#\xdd\xd5x\x1e\x...   
4  b'\xa4\x88\x1a(\x99tU\xff\xf60#\xdd\xd5x\x1e\x...   

                                        MMAE_DETAILS  
0  b'\x0c\xfa#|\x9b\xc0;\x16\xaf\xa5\x8f.\x95\xad...  
1  b"\xc2\xe7W6\xd0q>{\xaf\x9c\xd1\x15g\xa1uVA'\x...  
2  b'\x93\xbd\xd8\x87\xca3oNI\x9e\xb2B\xf4c\xb0[\...  
3  b'\x93\xbd\xd8\x87\xca3oNI\x9e\xb2B\xf4c\xb0[\...  
4  b"\xc2\xe7W6\xd0q>{\xaf\x9c\xd1\x15g\xa1uVA'\x...

# UPLOAD AND DOWNLOAD ENCRYPTED DF TO AND FROM AWS

In [22]:
# CREATE SFTP CONNECTION

import pysftp

dest_ip ="10.2.249.147"
ip2 ="10.2.249.169"

cnopts = pysftp.CnOpts(knownhosts=dest_ip)
cnopts.hostkeys = None

#Connect
srv = pysftp.Connection(host=dest_ip, username="mpesa-sftp",cnopts=cnopts, private_key="../main_keys/drc_pvt_key.pem", private_key_pass="mpesa")


In [23]:
# UPLOAD DATA
srv.put('drc_acct_entry_50.parquet','drc/drc_acct_entry_50.parquet')

<SFTPAttributes: [ size=38545 mode=0o100744 atime=1635410042 mtime=1635410042 ]>

In [24]:
# DOWNLOAD DATA
srv.get('drc/drc_acct_entry_50.parquet', 'drc_acct_entry_50_return.parquet')

In [25]:
df_encrypt2 = pd.read_parquet('drc_acct_entry_50_return.parquet')

In [26]:
df_encrypt2.head()

F_FI_ID   MMAE_LOAD_DATA_TS  MMAE_ACCOUNT_ENTRY_INDEX MMAE_TRANS_INDEX  \
0  763758329 2020-03-16 11:09:53                3887468029       7CD0RQEYOA   
1  763758329 2020-03-16 11:09:53                3887468038       7CD4RQEYOE   
2  763758329 2020-03-16 11:09:53                3887463928       7CD0RQEZAI   
3  763758329 2020-03-16 11:09:53                3887466508       7CD9RQF00B   
4  763758329 2020-03-16 11:09:53                3887466509       7CD9RQF00B   

  MMAE_RECORDTIMESTAMP REF_FT_ID MMAE_BUSINESS_SCOPE  F_RT_SEQID  REF_RT_ID  \
0  2020-03-13 08:27:32         2                   2         733       1237   
1  2020-03-13 08:27:33         2                   2         743       1278   
2  2020-03-13 08:27:33         1                   2         743       1278   
3  2020-03-13 08:27:33         1                   2         743       1278   
4  2020-03-13 08:27:33         2                   2         743       1278   

   REF_CT_ID  ...                       MMAE_TRANS_IDENTITY_MNEMONIC  \
0          2  ...  b'\x0c\xfa#|\x9b\xc0;\x16\xaf\xa5\x8f.\x95\xad...   
1          2  ...  b'\xc8\xedY\x92\x1dmK\xc0y6}\xa8\xcb\xaacqe\xb...   
2         29  ...  b'*\x17\xa9g\xfdC\xca\x0e3\x115\xa0N\xf3\x85\x02'   
3         29  ...  b'\xa4\x88\x1a(\x99tU\xff\xf60#\xdd\xd5x\x1e\x...   
4          2  ...  b'\xa4\x88\x1a(\x99tU\xff\xf60#\xdd\xd5x\x1e\x...   

  MMAE_ACCOUNT_VERSION MMAE_TRANSACTION_END_TIME MMAE_SUBJECT_ID  \
0                 1582       2020-03-13 08:27:33               0   
1                  370       2020-03-13 08:27:33               0   
2              1473965       2020-03-13 08:27:33               0   
3                 5830       2020-03-13 08:27:33               0   
4                 5864       2020-03-13 08:27:33               0   

   MMAE_AE_CREATE_TIME  MMAE_AE_AUDIT_INDEX  MMAE_RESERVE_BALANCE  \
0  2020-03-13 08:27:33           2923895500                     0   
1  2020-03-13 08:27:33           2923895526                     0   
2  2020-03-13 08:27:33           2923895532                     0   
3  2020-03-13 08:27:33           2923895538                     0   
4  2020-03-13 08:27:33           2923895539                     0   

   MMAE_PRE_RESERVE_BALANCE MMAE_PRE_AVAILABLE_BALANCE  F_FI_ID_ORIG  
0                         0                     676890            -1  
1                         0                     999687            -1  
2                         0                    2419243            -1  
3                         0                      64586            -1  
4                         0                     258415            -1  

[5 rows x 40 columns]

In [27]:
df_encrypt2[['MMAE_IDENTITY_MNEMONIC','MMAE_TRANS_IDENTITY_MNEMONIC','MMAE_DETAILS']].head()

MMAE_IDENTITY_MNEMONIC  \
0  b'\x0c\xfa#|\x9b\xc0;\x16\xaf\xa5\x8f.\x95\xad...   
1  b'\xc8\xedY\x92\x1dmK\xc0y6}\xa8\xcb\xaacqe\xb...   
2  b'*\x17\xa9g\xfdC\xca\x0e3\x115\xa0N\xf3\x85\x02'   
3  b'\xa4\x88\x1a(\x99tU\xff\xf60#\xdd\xd5x\x1e\x...   
4  b'\xa4\x88\x1a(\x99tU\xff\xf60#\xdd\xd5x\x1e\x...   

                        MMAE_TRANS_IDENTITY_MNEMONIC  \
0  b'\x0c\xfa#|\x9b\xc0;\x16\xaf\xa5\x8f.\x95\xad...   
1  b'\xc8\xedY\x92\x1dmK\xc0y6}\xa8\xcb\xaacqe\xb...   
2  b'*\x17\xa9g\xfdC\xca\x0e3\x115\xa0N\xf3\x85\x02'   
3  b'\xa4\x88\x1a(\x99tU\xff\xf60#\xdd\xd5x\x1e\x...   
4  b'\xa4\x88\x1a(\x99tU\xff\xf60#\xdd\xd5x\x1e\x...   

                                        MMAE_DETAILS  
0  b'\x0c\xfa#|\x9b\xc0;\x16\xaf\xa5\x8f.\x95\xad...  
1  b"\xc2\xe7W6\xd0q>{\xaf\x9c\xd1\x15g\xa1uVA'\x...  
2  b'\x93\xbd\xd8\x87\xca3oNI\x9e\xb2B\xf4c\xb0[\...  
3  b'\x93\xbd\xd8\x87\xca3oNI\x9e\xb2B\xf4c\xb0[\...  
4  b"\xc2\xe7W6\xd0q>{\xaf\x9c\xd1\x15g\xa1uVA'\x...

# DECRYPT DOWNLOADED DATA AND COMPARE TO ORIGINAL SQL IMPORT

In [32]:
# DECRYPT DATA

encrypt = Encrypt()

df_decrypt = encrypt.decrypt_df_columns(df_encrypt2,['MMAE_IDENTITY_MNEMONIC','MMAE_TRANS_IDENTITY_MNEMONIC','MMAE_DETAILS'],['str','str','str']
                                        ,'../main_keys/secret_hex_2021-10-28-11-18-48.secret')

In [33]:
df[['MMAE_IDENTITY_MNEMONIC','MMAE_TRANS_IDENTITY_MNEMONIC','MMAE_DETAILS']].head()

MMAE_IDENTITY_MNEMONIC  \
0           243818202547 - Francine Kapinga   
1  243820915425 - ROSETTE BATAKAMBA NTAMBUE   
2                            3218 - ETS MKA   
3       5137203 - YEMBA ONGENGE RARE5137203   
4       5137203 - YEMBA ONGENGE RARE5137203   

               MMAE_TRANS_IDENTITY_MNEMONIC  \
0           243818202547 - Francine Kapinga   
1  243820915425 - ROSETTE BATAKAMBA NTAMBUE   
2                            3218 - ETS MKA   
3       5137203 - YEMBA ONGENGE RARE5137203   
4       5137203 - YEMBA ONGENGE RARE5137203   

                                    MMAE_DETAILS  
0                243818202547 - Francine Kapinga  
1                         Deposit at Agent Store  
2  Deposit Commission received from Charge Payer  
3  Deposit Commission received from Charge Payer  
4                         Deposit at Agent Store

In [34]:
df_decrypt[['MMAE_IDENTITY_MNEMONIC','MMAE_TRANS_IDENTITY_MNEMONIC','MMAE_DETAILS']].head()

MMAE_IDENTITY_MNEMONIC  \
0           243818202547 - Francine Kapinga   
1  243820915425 - ROSETTE BATAKAMBA NTAMBUE   
2                            3218 - ETS MKA   
3       5137203 - YEMBA ONGENGE RARE5137203   
4       5137203 - YEMBA ONGENGE RARE5137203   

               MMAE_TRANS_IDENTITY_MNEMONIC  \
0           243818202547 - Francine Kapinga   
1  243820915425 - ROSETTE BATAKAMBA NTAMBUE   
2                            3218 - ETS MKA   
3       5137203 - YEMBA ONGENGE RARE5137203   
4       5137203 - YEMBA ONGENGE RARE5137203   

                                    MMAE_DETAILS  
0                243818202547 - Francine Kapinga  
1                         Deposit at Agent Store  
2  Deposit Commission received from Charge Payer  
3  Deposit Commission received from Charge Payer  
4                         Deposit at Agent Store

In [38]:
print('Issue Columns')
s = (df_decrypt == df).sum() / len(df)
s[s<1]

Issue Columns


REF_CTC_ID                    0.38
MMAE_UPDATE_BALANCE_STATUS    0.00
dtype: float64

# ADDITIONAL TESTS

In [ ]:
(encrypt.decrypt_df_columns(df_encrypt.tail(20),['RT_ID','RT_SEQID','RT_CODE'],['str','int','str']
                                        ,'secret_hex_2021-10-11-17-22-52.secret') == df.tail(20)).sum()

In [ ]:
df_encr_reduced.head()

In [ ]:
# DECRYPT DATA

encrypt = Encrypt()

df_decr_reduced = encrypt.decrypt_df_columns(df_encr_reduced,['id','seqid'],['str','int']
                                        ,'secret_hex_2021-10-11-17-22-52.secret')

In [ ]:
df_decr_reduced.head()

In [ ]:
(df.loc[300:600,'RT_ID'] == df_decr_reduced.id).all()

In [ ]:
(df.loc[300:600,'RT_SEQID'] == df_decr_reduced.seqid).all()

In [ ]:
(df.loc[300:600,'RT_SAMEAS_RT_SEQID'] == df_decr_reduced.sameas_seqid).all()